# 0. what for
1. 根据seed，保存mask idx

# 1.

In [49]:
# 人工数据集or真实数据集
ARTIFICIAL = False
seed = 277      # 3,31,204,223,277


noise_pct = 0.05  # 噪音占比:我们希望生成的data总体上最多浮动的百分比noise_pct


# Data path
if ARTIFICIAL:

    target_path = "../data/artificial_targets_v2_" + "noise=" + str(noise_pct)

else:

    target_path = "../data/targets_all"


# output path
val_mask_root_path = "../data/mask_idx/mask_idx_val_seed="+str(seed) + "/"
test_mask_root_path = "../data/mask_idx/mask_idx_test_seed="+str(seed)+ "/"
file_head = "mask_idx_"
file_tail= ".csv"

import numpy as np
import pandas as pd
import os
from metric_for_GTs_func import *
from tqdm import tqdm
import torch
import random


In [50]:
def setup_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True

setup_seed(seed)

# target_path里有全部的target data地址
target_all_path = os.listdir(target_path)
LEN_ALL = target_all_path.__len__()

print("seed=",seed)

seed= 277


In [51]:
if not os.path.exists(val_mask_root_path):
    os.makedirs(val_mask_root_path)
if not os.path.exists(test_mask_root_path):
    os.makedirs(test_mask_root_path)

In [52]:
col_names = ["mask_idx"]
mask_pct_val = 0.2
mask_pct_test = 0.1

threshold_len = 10        # len_i < 10不设置mask

In [53]:
for i in tqdm(range(LEN_ALL)):
# for i in tqdm(range(0,2)):
        target_path_i_path = os.path.join(target_path, target_all_path[i])
        target_df = pd.read_csv(target_path_i_path,encoding="utf-8")

        len_i = target_df.shape[0]          # data records amount

        if len_i<threshold_len:
            print("!!!!!!")
        n_mask = int(np.floor((mask_pct_test+mask_pct_val)*len_i))   # mask的总规模
        mask_list = sorted([random.randint(0, len_i) for i in range(n_mask)])

        val_mask = mask_list[0:int(np.floor(mask_pct_val*len_i))]          # val mask list
        test_mask = mask_list[len(val_mask):]                               # 剩下的是test

        val_mask_df = pd.DataFrame(val_mask,columns=col_names)
        test_mask_df = pd.DataFrame(test_mask,columns=col_names)

        # Save
        val_output_path = val_mask_root_path + file_head + str(i).zfill(4) + file_tail
        test_output_path = test_mask_root_path + file_head + str(i).zfill(4) + file_tail

        val_mask_df.to_csv(val_output_path,header=True,index=False,encoding="utf-8")
        test_mask_df.to_csv(test_output_path,header=True,index=False,encoding="utf-8")


100%|██████████| 1276/1276 [00:01<00:00, 737.88it/s]
